In [1]:
from transformers import pipeline
# class for tokenizer and sequence classification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

/opt/anaconda3/envs/learning/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# simple pretrained classifier
classifier = pipeline("sentiment-analysis")
res = classifier("How are you doing today?")
print(res)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9429818391799927}]


You can find different models [here]("")

from_pretrained(model_name): load pretrained model for any architecture

In [3]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
res = classifier("How are you doing today?")

observe that input_ids includes 101 and 102, which indicate the beginning and end respectively

In [4]:
tokens = tokenizer.tokenize("We are very happy to show you the huggingface transformers library")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer("We are very happy to show you the huggingface transformers library")

print("Tokens: %s" % tokens)
print("Token IDs: %s" % token_ids)
print("Input IDs: %s" % input_ids)


Tokens: ['we', 'are', 'very', 'happy', 'to', 'show', 'you', 'the', 'hugging', '##face', 'transformers', 'library']
Token IDs: [2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 17662, 12172, 19081, 3075]
Input IDs: {'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 17662, 12172, 19081, 3075, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Pad or truncate all sample points for consistency

In [5]:
X_train = ["We are very happy to show you the HuggingFace Transformers library",
            "We hope you don't hate it."]

batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [10]:
with torch.no_grad():
    # batch is a dictionary, so we unpack them
    outputs = model(**batch)
    print(outputs)
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=None, logits=tensor([[-4.0383,  4.2478],
        [ 0.0818, -0.0418]]), hidden_states=None, attentions=None)
tensor([[2.5194e-04, 9.9975e-01],
        [5.3086e-01, 4.6914e-01]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']
